In [ ]:
# https://github.com/Newmu/Theano-Tutorials
# see batch normalization

In [1]:
import datasets

datasets.downloads = '/home/greg/Downloads/'

trX, teX, trY, teY = datasets.mnist()

In [129]:
import numpy as np

In [146]:
X = trX
Y = trY

labels = np.argmax(Y, axis = 1)

epochs = 1
H = 625

N, D = X.shape
N, C = Y.shape

def accuracy(c):
    return np.mean(c == labels[:len(yhat)])

In [142]:
import tensorflow as tf

x = tf.placeholder('float')
y = tf.placeholder('float')

weights = lambda shape: tf.Variable(np.random.normal(size = shape).astype(np.float32))
params = []

w = weights([D, H])
b = weights([H])
l = tf.nn.relu(tf.matmul(x, w) + b)
params += [w, b]

w = weights([H, C])
b = weights([C])
model = tf.matmul(l, w) + b
params += [w, b]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model, y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    for _ in range(epochs):
        for start, end in zip(range(0, N, 100), range(100, N, 100)):
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

    params = [p.eval() for p in params]
    
relu = lambda x: x * (x > 0)
w1, b1, w2, b2 = params
c = np.argmax(relu(X[:1000].dot(w1) + b1).dot(w2) + b2, axis = 1)
print accuracy(c)

0.865


In [153]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(output_dim = H, input_dim = D))
model.add(Activation('relu'))
model.add(Dense(output_dim = C))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, Y, nb_epoch = epochs, batch_size = 100)
c = model.predict_classes(X[:1000])
print
print accuracy(c)

Epoch 1/1
 960/1000 [===========================>..] - ETA: 0s
0.967
